In [ ]:
###########################################################################
########################날씨 자료 가지고 오기##############################
###########################################################################

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/smubigdata2023')
from google.colab import files
files.upload()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv("output_replace_name.csv")
data.head()

,title,link,mountain,upload_to,운해 여부,위치
0,오늘 일출과 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘심항산’],20230508,1,충주
1,"노동절 새벽, 월출산의 운해일출",https://cafe.naver.com/windstopper/ArticleRead...,['월출산'],20230501,1,강진
2,백운대 일출산행(feat. 운해사냥),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230501,1,서울
3,백운대 운해사냥성공(thanks to또민아빠님),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230430,1,서울
4,지금 이 순간(역대급 제비봉 운해),https://cafe.naver.com/windstopper/ArticleRead...,[‘제비봉’],20230427,1,제천


In [ ]:
data['위치'] = data['위치'].replace('울주', '울산')
data['위치'] = data['위치'].replace('구례', '함양')
data['위치'] = data['위치'].replace('영암', '강진')
data['위치'] = data['위치'].replace('남양주', '양평')
data['위치'] = data['위치'].replace('단양', '제천')
data['위치'] = data['위치'].replace('무주', '금산')
data['위치'] = data['위치'].replace('과천', '수원')
data['위치'].value_counts()

함양    195
서울    185
인제     65
합천     61
제천     47
울산     33
부산     31
강진     28
양평     26
광주     24
금산     22
태백     22
충주     21
대구     20
파주     20
수원     19
Name: 위치, dtype: int64

In [ ]:
data['운해 여부'].value_counts()

0    514
1    305
Name: 운해 여부, dtype: int64

In [ ]:
# '위치' 열의 값 개수를 계산하여 새로운 열 '위치_개수'를 추가함
data['위치_개수'] = data.groupby('위치')['위치'].transform('count')

# '위치_개수'가 19 이상인 행들만 출력
data = data[data['위치_개수'] >= 19]

data = data.drop('위치_개수', axis=1)

In [ ]:
#api 호출해보기
location_code = {'함양': 264, '서울': 108, '인제':211, '합천': 285, '제천': 221, '울산': 152, '부산': 159, '강진': 259, '양평': 202, '광주': 156, '금산': 238, '태백': 216, '충주': 127, '대구': 143, '파주': 99, '수원': 119}
api = data[['upload_to', '위치']]
api['위치'] = api['위치'].map(location_code)
api

In [ ]:
import requests
import json

In [ ]:
from time import sleep
#데이터 가지고 오는 함수
i = 0
def get_weather(row):
  global i
  i+=1
  print(i)
  url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=2DkXXjLqs0UFB%2Bfol7F0H28uXY8PrJV9ujI%2FSCWai8az426V%2FtGZdtT8Wys6GdIcYVhwXLejRQwdVwNWdoPJ4Q%3D%3D&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt='+str(row['upload_to'])+ '&startHh=00&endDt='+str(row['upload_to'])+ '&endHh=05&stnIds='+str(int(row['위치']))
  try:
    response = requests.get(url)
    json_data = response.json()
    s1 = json.dumps(json_data)
    json_object = json.loads(s1)
    items = json_object['response']['body']['items']['item']
    night_data = items[0]
    sunrise_data = items[-1]
    night_ta = None
    if (night_data['ta'] != ''):
      night_ta = float(night_data['ta']) #기온

    night_rn = 0.0 
    if(night_data['rn'] != ''):
      night_rn = float(night_data['rn']) #강수량
    
    night_hm=None
    if (night_data['hm'] != ''):
      night_hm = float(night_data['hm']) #습도
    
    night_pa = None
    if (night_data['pa'] != ''):
      night_pa = float(night_data['pa']) #기압

    sunrise_ta = None
    if(sunrise_data['ta'] != ''):
      sunrise_ta = float(sunrise_data['ta']) #기온

    sunrise_ws = None #풍속
    if(sunrise_data['ws'] != ''):
      sunrise_ws = float(sunrise_data['ws']) 

    sunrise_hm = None
    if(sunrise_data['hm'] != ''):
      sunrise_hm = float(sunrise_data['hm']) #습도
    
    sunrise_pa = None
    if (sunrise_data['pa'] != ''):
      sunrise_pa = float(sunrise_data['pa']) #기압

    sunrise_dc10Tca = 0.0
    if(sunrise_data['dc10Tca'] != ''):
      sunrise_dc10Tca = float(sunrise_data['dc10Tca']) #전운량

      
  except Exception as e:
    print("실패")
    return pd.Series([None] * 9)

  return pd.Series([night_ta, night_rn, night_hm, night_pa, sunrise_ta, sunrise_ws, sunrise_hm, sunrise_pa, sunrise_dc10Tca])

In [ ]:
# get_weather(api.iloc[0])
new_columns = ['전날 기온', '전날 강수량', '전날 습도', '전날 기압', '일출 기온', '일출 풍속', '일출 습도', '일출 기압', '일출 전운량']
# sample = api.head()
# sample[new_columns] = sample.apply(get_weather, axis=1)
# sample
api[new_columns] = api.apply(get_weather, axis=1)

In [ ]:
api

,upload_to,위치,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량
0,20230508,127,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0
1,20230501,259,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0
2,20230501,108,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0
3,20230430,108,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0
4,20230427,221,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0
5,20221015,127,13.6,0.0,93.0,1005.3,10.6,0.8,96.0,1005.4,4.0
6,20230419,108,12.2,0.0,92.0,1002.3,12.1,2.1,92.0,1001.3,0.0
7,20230419,202,13.1,0.0,86.0,1008.7,12.2,0.4,88.0,1008.0,8.0
8,20230416,202,10.7,0.0,86.0,999.4,9.2,1.4,89.0,999.1,9.0
9,20230325,264,9.7,0.0,92.0,1001.6,8.6,0.7,96.0,1000.5,6.0


In [ ]:
api = api.drop(['upload_to', '위치'], axis=1)
data = pd.concat([data, api], axis=1)
data

,title,link,mountain,upload_to,운해 여부,위치,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량
0,오늘 일출과 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘심항산’],20230508,1,충주,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0
1,"노동절 새벽, 월출산의 운해일출",https://cafe.naver.com/windstopper/ArticleRead...,['월출산'],20230501,1,강진,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0
2,백운대 일출산행(feat. 운해사냥),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230501,1,서울,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0
3,백운대 운해사냥성공(thanks to또민아빠님),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230430,1,서울,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0
4,지금 이 순간(역대급 제비봉 운해),https://cafe.naver.com/windstopper/ArticleRead...,[‘제비봉’],20230427,1,제천,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0
5,지금 이 순간(운해 보러 온 동백이),https://cafe.naver.com/windstopper/ArticleRead...,[‘충주남산’],20221015,1,충주,13.6,0.0,93.0,1005.3,10.6,0.8,96.0,1005.4,4.0
6,현직 백운대 운해 관람중..,https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230419,1,서울,12.2,0.0,92.0,1002.3,12.1,2.1,92.0,1001.3,0.0
7,오늘 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘천마산'],20230419,1,양평,13.1,0.0,86.0,1008.7,12.2,0.4,88.0,1008.0,8.0
8,천마산 운해(?) 보고 왔어요⛰️,https://cafe.naver.com/windstopper/ArticleRead...,['천마산'],20230416,1,양평,10.7,0.0,86.0,999.4,9.2,1.4,89.0,999.1,9.0
9,오늘 천왕봉 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘천왕봉'],20230325,1,함양,9.7,0.0,92.0,1001.6,8.6,0.7,96.0,1000.5,6.0


In [ ]:
data.to_csv('final_crawled_api_data.csv', index=False)

In [ ]:
################################################################################
#########################예보 데이터 가지고 오기################################
################################################################################

In [1]:
import requests
import json
from datetime import datetime, timezone, timedelta
import pandas as pd

In [2]:
appid = "dd067a9aa888d5941ce0662190c0a2f4"
geo = {"함양": [35.52, 127.75], "서울": [37.54, 126.98], "인제": [38.06, 128.17], "합천": [35.50, 128.16], "제천": [37.13, 128.19], "울산": [35.53, 129.31], "부산": [35.17, 129.07], "강진": [34.80, 126.69], "양평": [37.21, 127.18], "광주": [35.15, 126.85], "금산": [36.03, 127.48], "태백": [37.16, 128.98], "충주": [36.99, 127.92], "대구": [35.87, 128.60], "파주": [37.75, 126.78], "수원": [37.26, 127.02]}
url = "http://api.openweathermap.org/data/2.5/forecast?lat="+str(geo["함양"][0])+"&lon="+str(geo["함양"][1])+"&appid="+ appid +"&units=metric"

In [3]:
response = requests.get(url)
json_data = response.json()
s1 = json.dumps(json_data)
json_object = json.loads(s1)
forecast_list = json_object['list']

In [4]:
days = []

KST = timezone(timedelta(hours=9))
now = datetime.now(KST)
days.append(now.strftime('%Y-%m-%d'))
for i in range(1, 5):
  day = now + timedelta(days=i)
  days.append(day.strftime('%Y-%m-%d'))
midnight = " 00:00:00"
sunrise = " 06:00:00"


In [10]:
df = pd.DataFrame(index=[1])
for forecast in forecast_list:
  if forecast['dt_txt'][:10] == days[1]: #임시로
    if forecast['dt_txt'][10:] == midnight:
      newColumns = ['전날 기온', '전날 강수량', '전날 습도', '전날 기압']
      temp = forecast['main']['temp']
      rain = 0
      humidity = forecast['main']['humidity']
      pressure = forecast['main']['pressure']
      if forecast['weather'][0]['main'] == 'Rain':
        rain = forecast['rain']['3h']
      df[newColumns] = pd.Series([temp, rain, humidity, pressure])
      continue
    if forecast['dt_txt'][10:] == sunrise:
      newColumns = ['일출 기온', '일출 풍속', '일출 습도', '일출 기압', '일출 전운량']
      temp = forecast['main']['temp']
      wind = forecast['wind']['speed']
      humidity = forecast['main']['humidity']
      pressure = forecast['main']['pressure']
      clouds = float(forecast['clouds']['all'])/10
      df[newColumns] = pd.Series([temp, wind, humidity, pressure, clouds])
      continue
# [night_ta, night_rn, night_hm, night_pa, sunrise_ta, sunrise_ws, sunrise_hm, sunrise_pa, sunrise_dc10Tca
df

,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량
1,24.22,0.0,56.0,1006.0,27.91,4.27,51.0,1005.0,0.7
